In [1]:
import requests
import socket
import ipinfo
import ssl
from cryptography import x509
from cryptography.hazmat.backends import default_backend

from urllib import parse, robotparser, request
import re
import json


ipinfo_api_key = '87ad97sa9dsa9asd'


#### Gets the IP adress of the given website

In [17]:
def ip_address(website):
    if "://" in website:
        website = website.split("://")[1]

    parts = website.split("/")
    domain = parts[0]
    try:
        ip_address = socket.gethostbyname(domain)
        return (domain, ip_address)
    except socket.gaierror:
        return (domain, "Invalid domain or unable to resolve IP address")


In [18]:
ip_address("example.com")

('example.com', '93.184.216.34')

#### Gets the IP information of the given IP address

In [11]:

def ip_info(ip_address):
    handler = ipinfo.getHandler(ipinfo_api_key)

    details = handler.getDetails(ip_address)
    return json.dumps(details.all)

#### Gets the SSL certificate of the given IP adress

In [91]:
def get_ssl(hostname, port=443):
    context = ssl.create_default_context()
    conn = context.wrap_socket(socket.socket(socket.AF_INET, socket.SOCK_STREAM), server_hostname=hostname)
    conn.connect((hostname, port))

    certs = conn.getpeercert(True)
    certificate = x509.load_der_x509_certificate(certs, default_backend())
    subject = next((attr.value for attr in certificate.subject if attr.oid == x509.NameOID.COMMON_NAME), None)
    issuer = next((attr.value for attr in certificate.issuer if attr.oid == x509.NameOID.ORGANIZATION_NAME ), None)
    certificate_info = {
        "subject": subject,
        "issuer": issuer,
        "serial_number": certificate.serial_number,
        "not_valid_before": certificate.not_valid_before.isoformat(),
        "not_valid_after": certificate.not_valid_after.isoformat()
    }
    print(type(certificate_info))
    return json.dumps(certificate_info)


#### Reverse DNS

In [ ]:
def get_records(domain):
    results = {}
    record_types = ['NONE', 'A', 'NS', 'MD', 'MF', 'CNAME', 'SOA', 'MB', 'MG', 'MR', 'NULL', 'WKS', 'PTR', 'HINFO', 'MINFO', 'MX', 'TXT', 'RP', 'AFSDB', 'X25', 'ISDN', 'RT', 'NSAP', 'NSAP-PTR', 'SIG', 'KEY', 'PX', 'GPOS', 'AAAA', 'LOC', 'NXT', 'SRV', 'NAPTR', 'KX', 'CERT', 'A6', 'DNAME', 'OPT', 'APL', 'DS', 'SSHFP', 'IPSECKEY', 'RRSIG', 'NSEC', 'DNSKEY', 'DHCID', 'NSEC3', 'NSEC3PARAM', 'TLSA', 'HIP', 'CDS', 'CDNSKEY', 'CSYNC', 'SPF', 'UNSPEC', 'EUI48', 'EUI64', 'TKEY', 'TSIG', 'IXFR', 'AXFR', 'MAILB', 'MAILA', 'ANY', 'URI', 'CAA', 'TA', 'DLV']
    
    for record_type in record_types:
        try:
            answers = dns.resolver.resolve(domain, record_type)
            results[record_type] = [rdata.to_text() for rdata in answers]
        except dns.resolver.NoAnswer:
            pass
        except dns.resolver.NXDOMAIN:
            pass
        except Exception as e:
            continue
    
    return json.dumps(results)

#### Sitemap Parser

In [ ]:
def get_sitemaps(website):
    robotstxturl = parse.urljoin(website, "robots.txt")
    try:
        rp = robotparser.RobotFileParser()
        rp.set_url(robotstxturl)
        rp.read()
        sitemaps = rp.site_maps()
    except robotparser.RobotFileParserError as e:
        print(f"error: {e}")
    except Exception as e:
        print(f"Error: {e}")
        
    return sitemaps

def sitemap_parser(sitemap):
    r = request.urlopen(sitemap)
    xml = r.read().decode('utf8')
    elements = re.findall(r'<loc>(.*?)<\/loc>', xml, re.DOTALL)

    urls = []

    for element in elements:
        if element.endswith('.xml'):
            urls.extend(sitemap_parser(element))  # Recursively call sitemap_parser
        else:
            urls.append(element)

    return urls

def site_maps(url):
    sitemaps = get_sitemaps(url)
    
    all_urls = []
    
    for sitemap in sitemaps:
        all_urls.extend(sitemap_parser(sitemap))

    urls_json = json.dumps(all_urls)
    
    return(urls_json)

In [ ]:
def get_host(domain):
    

In [40]:
import socket
import dns.resolver

def gather_domains_and_subdomains(primary_domain):
    domains = set()
    subdomains = set()

    try:
        ip_address = socket.gethostbyname(primary_domain)

        answers = dns.resolver.resolve(primary_domain, 'A')


        answers = dns.resolver.resolve(primary_domain, 'NS')
        for rdata in answers:
            domains.add(rdata.to_text())
        print(domains)

    except (socket.gaierror, dns.resolver.NXDOMAIN, dns.resolver.NoAnswer) as e:
        print(f"Error: {e}")


primary_domain = 'example.com'  # Replace with the domain you want to check
gather_domains_and_subdomains(primary_domain)


{'b.iana-servers.net.', 'a.iana-servers.net.'}


#### Headers

In [10]:
import requests

def get_headers(domain):
    response = requests.get(domain)
    headers = response.headers
    headers_dict = {header: value for header,value in headers.items()}
    return json.dumps(headers_dict)
get_headers('http://dsu.edu')

'{"Content-Length": "28226", "Content-Type": "text/html", "Date": "Fri, 27 Oct 2023 21:58:56 GMT", "Server": "Microsoft-IIS/10.0", "Accept-Ranges": "bytes", "Content-Encoding": "gzip", "ETag": "\\"0d545d0f88da1:0\\"", "Last-Modified": "Fri, 27 Oct 2023 17:12:50 GMT", "Set-Cookie": "ARRAffinity=7f02473c44e480611007b41ddc4a571fda96d68a199b0df5e41dfe5a3816077b;Path=/;HttpOnly;Secure;Domain=dsu.edu, ARRAffinitySameSite=7f02473c44e480611007b41ddc4a571fda96d68a199b0df5e41dfe5a3816077b;Path=/;HttpOnly;SameSite=None;Secure;Domain=dsu.edu", "Vary": "Accept-Encoding", "Strict-Transport-Security": "max-age=31536000", "X-Powered-By": "ASP.NET", "X-Frame-Options": "SAMEORIGIN", "X-Content-Type-Options": "nosniff"}'

#### Cookies

In [6]:

def get_cookies(domain):
    response = requests.get(domain)
    cookies = response.cookies
    cookies_dict = {cookie.name: cookie.value for cookie in cookies}
    return json.dumps(cookies_dict)
get_cookies('http://dsu.edu')


'{"ARRAffinity": "7f02473c44e480611007b41ddc4a571fda96d68a199b0df5e41dfe5a3816077b", "ARRAffinitySameSite": "7f02473c44e480611007b41ddc4a571fda96d68a199b0df5e41dfe5a3816077b"}'

In [47]:
import requests

def get_redirects(url):
    try:
        response = requests.get(url, allow_redirects=True)
        redirects = response.history
        final_url = response.url
        return redirects, final_url
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return [], None

if __name__ == '__main__':
    target_url = 'http://example.com'  # Replace with the URL you want to check
    redirects, final_url = get_redirects(target_url)

    if redirects:
        print("Redirects:")
        for index, response in enumerate(redirects, start=1):
            print(f"{index}. {response.url}")
        print("\nFinal URL:")
        print(final_url)
    else:
        print("No redirects found.")


<Response [200]>
No redirects found.


In [49]:
response = requests.get(url, allow_redirects=True)

for i in respose

SyntaxError: invalid syntax (1835790732.py, line 1)